## Projeto

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane.optimize import AdamOptimizer
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from tqdm.notebook import tqdm, trange

In [2]:

# Read the HDF5 file using pandas
data_frame_fcnc = pd.read_hdf('fcnc_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_fcnc.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_fcnc.head()


Number of rows: 90548


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,1.408853,15.150869,339.182312,2.350262,1.396943,0.710451,0.109013,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,test,7.762202e-09,0.001285
1,1,-2.481838,7.208333,247.036240,-2.280740,0.428710,0.205213,0.000000,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,val,7.762202e-09,0.001285
2,0,1.476267,94.220718,238.014694,-1.788097,94.256210,2.418446,1.585315,1.127324,0.431098,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
3,1,0.647480,13.459283,230.971832,-1.032663,1.227122,0.467150,0.164008,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
4,0,2.106436,97.490242,698.399902,-3.059983,36.555862,2.937936,1.799140,1.093004,0.589724,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285


In [3]:
# Read the HDF5 file using pandas
data_frame_bkg = pd.read_hdf('bkg_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_bkg.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_bkg.head()


Number of rows: 1002490


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,-1.988600,52.710262,229.350952,0.728242,36.148926,23.039709,16.949991,14.424411,12.000529,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,train,0.000018,11.9635
1,0,0.528382,61.115589,315.538910,-0.863614,32.592808,22.366640,16.285843,13.938633,11.180016,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,test,0.000018,11.9635
2,0,1.479911,98.012802,251.109573,-3.133624,90.252274,33.646885,30.612156,27.973904,23.729696,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
3,2,0.926899,-0.000007,240.909348,0.835656,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
4,0,0.781194,72.234299,206.020386,-0.320449,48.886372,20.743645,16.572512,13.070706,11.269534,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635


In [4]:
# Z-score normalization

# copy the data to a new dataframe
data_frame_fcnc_norm = data_frame_fcnc.copy()
data_frame_bkg_norm = data_frame_bkg.copy()

# normalize the data except the categorical features and the weights
for feature in data_frame_fcnc.columns:
    if feature in ['gen_decay_filter', 'gen_filter', 'gen_label', 'gen_n_btags', 'gen_sample', 'gen_sample_filter', 'gen_split', 'gen_decay2','gen_decay1', 'gen_xsec']:
        pass
    else: 
        data_frame_fcnc_norm[feature] = (data_frame_fcnc[feature] - data_frame_fcnc[feature].mean()) / data_frame_fcnc[feature].std()
        
for feature in data_frame_bkg.columns:
    if feature in ['gen_decay_filter', 'gen_filter', 'gen_label', 'gen_n_btags', 'gen_sample', 'gen_sample_filter', 'gen_split','gen_decay2','gen_decay1','gen_xsec']:
        pass
    else:  
        data_frame_bkg_norm[feature] = (data_frame_bkg[feature] - data_frame_bkg[feature].mean()) / data_frame_bkg[feature].std()

### VQC

#### Pennylane (V1)

In [5]:
# Split the original data into train, validation and test sets for each dataset
data_frame_fcnc_train = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'train']
data_frame_bkg_train =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'train']

data_frame_fcnc_test = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'test']
data_frame_bkg_test =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'test']

data_frame_fcnc_val = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'val']
data_frame_bkg_val =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'val']


# get 500 points of each dataset and  join the datasets (randomly)
train_fcnc = data_frame_fcnc_train.sample(n=500, random_state=42)
train_bkg = data_frame_bkg_train.sample(n=500, random_state=42)
train = pd.concat([train_fcnc, train_bkg])
train = train.sample(frac=1, random_state=42)

test_fcnc = data_frame_fcnc_test.sample(n=500, random_state=42)
test_bkg = data_frame_bkg_test.sample(n=500, random_state=42)
test= pd.concat([test_fcnc, test_bkg])
test = test.sample(frac=1,random_state=42)

val_fnc = data_frame_fcnc_val.sample(n=500, random_state=42)
val_bkg = data_frame_bkg_val.sample(n=500, random_state=42)
val = pd.concat([val_fnc, val_bkg])
val = val.sample(frac=1, random_state=42)

# get the weights for each dataset
w_train = train[['gen_xsec']]
w_test = test[['gen_xsec']]
w_val = val[['gen_xsec']]


# change the signal and bkg labels to 0 and 1 and get the labels for each dataset
train = train.replace(['signal'], 1)
train= train.replace(['bkg'], 0)
y_train = train[['gen_label']]
x_train = train[['MissingET_MET', 'Jet1_BTag']]

test = test.replace(['signal'], 1)
test= test.replace(['bkg'], 0)
y_test = test[['gen_label']]
x_test = test[['MissingET_MET', 'Jet1_BTag']]

val = val.replace(['signal'], 1)
val= val.replace(['bkg'], 0)
y_val = val[['gen_label']]
x_val = val[['MissingET_MET', 'Jet1_BTag']]

y_train_arr = np.concatenate(y_train.values, axis=0 )
y_val_arr = np.concatenate( y_val.values, axis=0 )
y_test_arr = np.concatenate( y_test.values, axis=0 )

# Renormalize data weights
w_train[y_train_arr == 1] = (w_train[y_train_arr == 1] / w_train[y_train_arr == 1].sum()) * y_train_arr.shape[0] / 2
w_train[y_train_arr == 0] = (w_train[y_train_arr == 0] / w_train[y_train_arr == 0].sum()) * y_train_arr.shape[0] / 2
        
w_test[y_test_arr == 1] = (w_test[y_test_arr == 1] / w_test[y_test_arr == 1].sum()) * w_test.shape[0] / 2
w_test[y_test_arr == 0] = (w_test[y_test_arr == 0] / w_test[y_test_arr == 0].sum()) * w_test.shape[0] / 2
        
w_val[y_test_arr == 1] = (w_val[y_test_arr == 1] / w_val[y_test_arr == 1].sum()) * w_val.shape[0] / 2
w_val[y_test_arr == 0] = (w_val[y_test_arr == 0] / w_val[y_test_arr == 0].sum()) * w_val.shape[0] / 2

# Concatenate features
X = np.concatenate([x_train, x_test,x_val])

# Normalize the data for angle embeding  (Put the data between -pi and pi)
X = (((X - X.min()) / (X.max() - X.min())) * 2 - 1) * (np.pi)

# Split the features array into train, validation and test sets
x_train = X[:1000]
x_test = X[1000:2000]
x_val = X[2000:]

In [6]:
# accuracy function
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


In [7]:
# loss function
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

# classifier function    
def classifier(n_features, n_layers, weights, x):
        #c = circuit(n_features, n_layers, weights, x)
        dev=qml.device("default.qubit", wires=n_features)
        return qml.QNode(circuit, dev)(n_features, n_layers, weights, x)
    
# cost function    
def cost(n_features, n_layers,weights,X,Y,W):  
        # Compute predictions
        y_scores = [(classifier(n_features, n_layers,weights, x) + 1) / 2 for x in X]

        loss = square_loss(Y, y_scores)
        loss = loss * W
        loss = loss.sum()
        
        return loss
    
# train step function    
def train_step(n_features, n_layers,x_train,y_train, w_train, weights, opt,desc='Training'):
        
        # Only require grad if necessary
        x_train = np.array(x_train, requires_grad=False)
        y_train = np.array(y_train, requires_grad=True)
        w_train = np.array(w_train, requires_grad=False)

        # Compute cost and update weights
        weights, loss = opt.step_and_cost(cost, n_features, n_layers,weights, X=x_train, Y=y_train, W=w_train)

        return loss, weights
    
# validation step function
def validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch,best_weights,desc='Validation'):
    X_val = np.array(x_val, requires_grad=False)
    Y_val = np.array(y_val, requires_grad=False)
    W_val = np.array(w_val, requires_grad=False)

    y_scores = np.array([classifier(n_features, n_layers, weights, x) for x in X_val])
    y_scores = (y_scores + 1) / 2

    W_val[Y_val == 1] = (W_val[Y_val == 1] / W_val[Y_val == 1].sum()) * W_val.shape[0] / 2
    W_val[Y_val == 0] = (W_val[Y_val == 0] / W_val[Y_val == 0].sum()) * W_val.shape[0] / 2

    auc_score = roc_auc_score(y_true=Y_val, y_score=y_scores, sample_weight=W_val)
    loss = cost(n_features, n_layers, weights, X_val, Y_val, W_val)


    if best_score is None or auc_score > best_score:
        best_score = auc_score
        best_score_epoch = epoch_number
        best_weights = weights

    tqdm.write(f"Epoch: {epoch_number}, Validation Loss: {loss:.4f}, AUC Score: {auc_score:.4f}")

    return best_score, best_score_epoch, best_weights
        
        
# train function
def train(n_features, n_layers, x_train, y_train, learning_rate, weights, max_epochs, epoch_number):
    opt = AdamOptimizer(learning_rate)
    best_score = None
    best_weights = None
    best_score_epoch = None

    with tqdm(total=max_epochs, desc='Epoch', unit='epoch') as pbar:
        for epoch in range(epoch_number, max_epochs):
            epoch_number = epoch

            loss, nf_nl_weights = train_step(n_features, n_layers, x_train, y_train, w_train, weights, opt, desc='Training')
            
            # Log variable values using tqdm.write
            tqdm.write(f"Epoch: {epoch_number:}, Loss: {loss:.4f}")
            
            
            weights = nf_nl_weights[2:]
            weights = weights[0]

            if epoch_number == max_epochs - 1 or (epoch_number+1)%5==0:
                best_score, best_score_epoch, best_weights = validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch, best_weights,desc='Validation')
                # early stopping
                if epoch_number - best_score_epoch > 30 and epoch_number > 80:
                    tqdm.write(f"Early stopping at epoch {epoch_number}")
                    break

            pbar.update(1)  # Update progress bar
        tqdm.write(f"Best Score: {best_score:.4f}")            
        
    return best_score, best_weights

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    

### Original

In [8]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)


Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 507.7428
Epoch: 1, Loss: 507.3548
Epoch: 2, Loss: 506.5597
Epoch: 3, Loss: 505.2399
Epoch: 4, Loss: 503.3643
Epoch: 4, Validation Loss: 495.7487, AUC Score: 0.2793
Epoch: 5, Loss: 500.9158
Epoch: 6, Loss: 497.8834
Epoch: 7, Loss: 494.2648
Epoch: 8, Loss: 490.0683
Epoch: 9, Loss: 485.3122
Epoch: 9, Validation Loss: 475.1420, AUC Score: 0.2793
Epoch: 10, Loss: 480.0247
Epoch: 11, Loss: 474.2452
Epoch: 12, Loss: 468.0227
Epoch: 13, Loss: 461.4143
Epoch: 14, Loss: 454.4808
Epoch: 14, Validation Loss: 442.9057, AUC Score: 0.2794
Epoch: 15, Loss: 447.2784
Epoch: 16, Loss: 439.8456
Epoch: 17, Loss: 432.2018
Epoch: 18, Loss: 424.3597
Epoch: 19, Loss: 416.3302
Epoch: 19, Validation Loss: 404.5329, AUC Score: 0.2796
Epoch: 20, Loss: 408.1271
Epoch: 21, Loss: 399.7712
Epoch: 22, Loss: 391.2895
Epoch: 23, Loss: 382.7125
Epoch: 24, Loss: 374.0721
Epoch: 24, Validation Loss: 362.7679, AUC Score: 0.2804
Epoch: 25, Loss: 365.4023
Epoch: 26, Loss: 356.7394
Epoch: 27, Loss: 348.1224
Epoc

In [9]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7896380941989091


#### Type of rotation in  angle embeding

In [10]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [11]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 506.2405
Epoch: 1, Loss: 503.8364
Epoch: 2, Loss: 501.0658
Epoch: 3, Loss: 497.9401
Epoch: 4, Loss: 494.4692
Epoch: 4, Validation Loss: 485.4169, AUC Score: 0.2797
Epoch: 5, Loss: 490.6675
Epoch: 6, Loss: 486.5476
Epoch: 7, Loss: 482.1270
Epoch: 8, Loss: 477.4251
Epoch: 9, Loss: 472.4609
Epoch: 9, Validation Loss: 462.4356, AUC Score: 0.2805
Epoch: 10, Loss: 467.2602
Epoch: 11, Loss: 461.8482
Epoch: 12, Loss: 456.2530
Epoch: 13, Loss: 450.5069
Epoch: 14, Loss: 444.6406
Epoch: 14, Validation Loss: 434.6423, AUC Score: 0.2818
Epoch: 15, Loss: 438.6886
Epoch: 16, Loss: 432.6849
Epoch: 17, Loss: 426.6627
Epoch: 18, Loss: 420.6546
Epoch: 19, Loss: 414.6898
Epoch: 19, Validation Loss: 405.7630, AUC Score: 0.2839
Epoch: 20, Loss: 408.7952
Epoch: 21, Loss: 402.9925
Epoch: 22, Loss: 397.2984
Epoch: 23, Loss: 391.7240
Epoch: 24, Loss: 386.2738
Epoch: 24, Validation Loss: 378.9335, AUC Score: 0.2862
Epoch: 25, Loss: 380.9458
Epoch: 26, Loss: 375.7326
Epoch: 27, Loss: 370.6212
Epoc

In [12]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7916381223677037


#### Retirar rotações

In [13]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            #W1 = weights[layer]

            # Define Rotations
            #for i in range(0,n_features):
                #qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [14]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

/home/brunasalgado/LIP/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 0, Loss: 507.7397
Epoch: 1, Loss: 507.7397
Epoch: 2, Loss: 507.7397
Epoch: 3, Loss: 507.7397
Epoch: 4, Loss: 507.7397
Epoch: 4, Validation Loss: 502.4745, AUC Score: 0.2793


/home/brunasalgado/LIP/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 5, Loss: 507.7397
Epoch: 6, Loss: 507.7397
Epoch: 7, Loss: 507.7397
Epoch: 8, Loss: 507.7397
Epoch: 9, Loss: 507.7397
Epoch: 9, Validation Loss: 502.4745, AUC Score: 0.2793
Epoch: 10, Loss: 507.7397
Epoch: 11, Loss: 507.7397
Epoch: 12, Loss: 507.7397
Epoch: 13, Loss: 507.7397
Epoch: 14, Loss: 507.7397


...

Epoch: 74, Validation Loss: 502.4745, AUC Score: 0.2793


/home/brunasalgado/LIP/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 75, Loss: 507.7397
Epoch: 76, Loss: 507.7397
Epoch: 77, Loss: 507.7397
Epoch: 78, Loss: 507.7397
Epoch: 79, Loss: 507.7397
Epoch: 79, Validation Loss: 502.4745, AUC Score: 0.2793
Epoch: 80, Loss: 507.7397
Epoch: 81, Loss: 507.7397
Epoch: 82, Loss: 507.7397
Epoch: 83, Loss: 507.7397
Epoch: 84, Loss: 507.7397


/home/brunasalgado/LIP/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 84, Validation Loss: 502.4745, AUC Score: 0.2793
Early stopping at epoch 84
Best Score: 0.2793


In [15]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.19866373599080078


### Retirar entanglement

In [16]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)


        return qml.expval(qml.PauliZ(0))

In [17]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 335.3266
Epoch: 1, Loss: 329.6770
Epoch: 2, Loss: 323.9578
Epoch: 3, Loss: 318.1685
Epoch: 4, Loss: 312.3315
Epoch: 4, Validation Loss: 306.2880, AUC Score: 0.7205
Epoch: 5, Loss: 306.4746
Epoch: 6, Loss: 300.6071
Epoch: 7, Loss: 294.7403
Epoch: 8, Loss: 288.9003
Epoch: 9, Loss: 283.1130
Epoch: 9, Validation Loss: 277.5215, AUC Score: 0.7201
Epoch: 10, Loss: 277.3929
Epoch: 11, Loss: 271.7552
Epoch: 12, Loss: 266.2254
Epoch: 13, Loss: 260.8283
Epoch: 14, Loss: 255.5800
Epoch: 14, Validation Loss: 251.1005, AUC Score: 0.7188
Epoch: 15, Loss: 250.4971
Epoch: 16, Loss: 245.6025
Epoch: 17, Loss: 240.9173
Epoch: 18, Loss: 236.4564
Epoch: 19, Loss: 232.2330
Epoch: 19, Validation Loss: 229.4415, AUC Score: 0.7168
Epoch: 20, Loss: 228.2627
Epoch: 21, Loss: 224.5593
Epoch: 22, Loss: 221.1309
Epoch: 23, Loss: 217.9824
Epoch: 24, Loss: 215.1188
Epoch: 24, Validation Loss: 214.3145, AUC Score: 0.7152
Epoch: 25, Loss: 212.5424
Epoch: 26, Loss: 210.2510
Epoch: 27, Loss: 208.2384
Epoc

In [18]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7980099854812013


#### Substituir CNOTs por CZ

In [19]:
# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CZ(wires=[i, 0])
                        else:
                            qml.CZ(wires=[i, i + 1])
                else:
                    qml.CZ(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [20]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 335.9112
Epoch: 1, Loss: 335.3076
Epoch: 2, Loss: 334.2567
Epoch: 3, Loss: 332.7333
Epoch: 4, Loss: 330.7300
Epoch: 4, Validation Loss: 327.9756, AUC Score: 0.7207
Epoch: 5, Loss: 328.2453
Epoch: 6, Loss: 325.2830
Epoch: 7, Loss: 321.8530
Epoch: 8, Loss: 317.9717
Epoch: 9, Loss: 313.6623
Epoch: 9, Validation Loss: 308.8147, AUC Score: 0.7207
Epoch: 10, Loss: 308.9548
Epoch: 11, Loss: 303.8856
Epoch: 12, Loss: 298.4986
Epoch: 13, Loss: 292.8441
Epoch: 14, Loss: 286.9782
Epoch: 14, Validation Loss: 281.0779, AUC Score: 0.7203
Epoch: 15, Loss: 280.9615
Epoch: 16, Loss: 274.8579
Epoch: 17, Loss: 268.7332
Epoch: 18, Loss: 262.6537
Epoch: 19, Loss: 256.6839
Epoch: 19, Validation Loss: 251.4132, AUC Score: 0.7195
Epoch: 20, Loss: 250.8838
Epoch: 21, Loss: 245.3077
Epoch: 22, Loss: 240.0029
Epoch: 23, Loss: 235.0080
Epoch: 24, Loss: 230.3518
Epoch: 24, Validation Loss: 227.1272, AUC Score: 0.7174
Epoch: 25, Loss: 226.0539
Epoch: 26, Loss: 222.1255
Epoch: 27, Loss: 218.5703
Epoc

In [21]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.801093015307948


#### Medir o segundo qubit

In [22]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(1))

In [23]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 499.7783
Epoch: 1, Loss: 499.2422
Epoch: 2, Loss: 498.3869
Epoch: 3, Loss: 497.2039
Epoch: 4, Loss: 495.6824
Epoch: 4, Validation Loss: 493.8151, AUC Score: 0.7071
Epoch: 5, Loss: 493.8138
Epoch: 6, Loss: 491.5871
Epoch: 7, Loss: 488.9929
Epoch: 8, Loss: 486.0243
Epoch: 9, Loss: 482.6725
Epoch: 9, Validation Loss: 478.9228, AUC Score: 0.6997
Epoch: 10, Loss: 478.9340
Epoch: 11, Loss: 474.8051
Epoch: 12, Loss: 470.2845
Epoch: 13, Loss: 465.3750
Epoch: 14, Loss: 460.0804
Epoch: 14, Validation Loss: 454.3825, AUC Score: 0.6937
Epoch: 15, Loss: 454.4085
Epoch: 16, Loss: 448.3712
Epoch: 17, Loss: 441.9819
Epoch: 18, Loss: 435.2601
Epoch: 19, Loss: 428.2268
Epoch: 19, Validation Loss: 420.8775, AUC Score: 0.6894
Epoch: 20, Loss: 420.9091
Epoch: 21, Loss: 413.3357
Epoch: 22, Loss: 405.5409
Epoch: 23, Loss: 397.5608
Epoch: 24, Loss: 389.4359
Epoch: 24, Validation Loss: 381.1884, AUC Score: 0.6829
Epoch: 25, Loss: 381.2067
Epoch: 26, Loss: 372.9172
Epoch: 27, Loss: 364.6095
Epoc

In [24]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.8021587958254714
